In [ ]:
import numpy as np
import mlrose_hiive
import pandas as pd
# Import sklearn 
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix, ConfusionMatrixDisplay
# Plotting tools
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
# Misc tools
import time

In [ ]:
# Make the figures bigger
plt.rcParams.update({'font.size': 20})

In [ ]:
# Import the dataset
# Load the data
heart_df = pd.read_csv('./heart_2.csv')
heart_X = heart_df.drop(columns =['target'])
heart_y = heart_df['target']

# Split the data into 30:70 ratio
X_train_heart, X_test_heart, y_train_heart, y_test_heart = train_test_split(heart_X, heart_y,
                                                    test_size = 0.3, random_state = 42, stratify = heart_y
                                                   )

# From before we should perform some scaling first
# We will be using a min max scaler to scale the data
scaler = MinMaxScaler()
scaler.fit(X_train_heart)
X_train_heart = scaler.transform(X_train_heart)
X_test_heart = scaler.transform(X_test_heart)

## RHC

In [ ]:
%%time
grid_search_parameters = ({
  'learning_rate_init': [0.0001,0.001,0.01,0.1,0.5],
  'restarts': [5],
  'activation': [mlrose_hiive.neural.activation.relu],
})
rhc_nn = mlrose_hiive.NNGSRunner(X_train_heart, y_train_heart, 
                                 X_test_heart, y_test_heart, 
                                 experiment_name= "NeuralNetworkRHC",
                                 output_directory="nn_rhc_algorithm/",
                                 seed=42, iteration_list =[10], 
                                 algorithm=mlrose_hiive.random_hill_climb,
                                 hidden_layer_sizes=[[100, 100]],
                                 grid_search_parameters=grid_search_parameters, 
                                 grid_search_scorer_method=f1_score,
                                 early_stopping = True,
                                 max_attempts =100,
                                 generate_curves = True,
                                 n_jobs=-1,
                                 cv=5
                                )
run_stats, curves, cv_results, best_est = rhc_nn.run()

In [ ]:
best_est.best_params_

In [ ]:
%%time
rhc_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'random_hill_climb', 
    max_iters=100000,
    learning_rate =  0.001,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    restarts = 5,
)

train_df_rhc = pd.DataFrame([])
val_df_rhc = pd.DataFrame([])

for i in range(5):
    print(i)
    X_train_train, X_test_train, y_train_train, y_test_train = train_test_split(X_train_heart, y_train_heart,
                                                        test_size = 0.2, random_state = (i+1)*i, stratify = y_train_heart
                                                       )

    rhc_nn_epochs.fit(X_train_train, y_train_train)
    train_curve = rhc_nn_epochs.fitness_curve[:,0]
    d = {i: train_curve}
    loss_df = pd.DataFrame(data=d)
    train_df_rhc = pd.concat([train_df_rhc,loss_df],1)
    rhc_nn_epochs.fit(X_test_train, y_test_train)
    val_curve = rhc_nn_epochs.fitness_curve[:,0]
    d = {i: val_curve}
    loss_df = pd.DataFrame(data=d)
    val_df_rhc = pd.concat([val_df_rhc,loss_df],1)

In [ ]:
train_df_rhc['Mean'] = train_df_rhc.loc[:, [0,1,2,3,4]].mean(axis = 1)
val_df_rhc['Mean'] = val_df_rhc.loc[:, [0,1,2,3,4]].mean(axis = 1)
ax = plt.gca()
plt.plot(train_df_rhc['Mean'],  label="Training Error")
plt.plot(val_df_rhc['Mean'], label ='Validation Error')
plt.title('Loss Curves - '+ 'RHC',fontsize = 16)
plt.xlabel("Epochs"), plt.ylabel("Loss Values"), plt.legend(loc="best",prop={'size': 14})
plt.show()

In [ ]:
%%time
rhc_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'random_hill_climb', 
    max_iters=100000,
    learning_rate =  0.001,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    restarts = 5,
)

time_start = time.time()
rhc_train = rhc_nn_epochs.fit(X_train_heart, y_train_heart)
fit_time = time.time()
print(f'fit_time = {fit_time-time_start}')
print(classification_report(y_test_heart, rhc_nn_epochs.predict(X_test_heart)))
print(classification_report(y_train_heart, rhc_nn_epochs.predict(X_train_heart)))
plt.plot(rhc_nn_epochs.fitness_curve[:,0],label='Training')
rhc_test = rhc_nn_epochs.fit(X_test_heart, y_test_heart)
plt.plot(rhc_nn_epochs.fitness_curve[:,0],label='Validation')
plt.legend()
fig=plt.gcf()
fig.set_size_inches(10,6)
plt.xlabel("Epoch", fontsize=18)
plt.ylabel("Loss", fontsize=18)

In [ ]:
y_pred = rhc_train.predict(X_test_heart)
cm = confusion_matrix(y_test_heart, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=['No','Yes'],
                             )
disp.plot()
disp.im_.colorbar.remove()
plt.title('RHC - Confusion Matrix')
plt.show()

## Genetic Algorithm

In [ ]:
%%time

grid_search_ga = {
    
    "learning_rate_init": [0.0001,0.001,0.01,0.1,0.5],
    "activation": [mlrose_hiive.neural.activation.relu],
    "population_sizes":[20,50,100,300],
    "mutation_rate":[0.1, 0.25, 0.5, 0.8], 
}

ga_nn = mlrose_hiive.NNGSRunner(X_train_heart, y_train_heart, 
                                 X_test_heart, y_test_heart, 
                                 experiment_name= "NeuralNetworkGA",
                                 output_directory="nn_ga_algorithm/",
                                 seed=42, iteration_list=[100000], 
                                 algorithm=mlrose_hiive.genetic_alg,
                                 hidden_layer_sizes=[[100, 100]],
                                 grid_search_parameters=grid_search_ga,
                                 grid_search_scorer_method=f1_score,
                                 early_stopping = True,
                                 max_attempts =100,
                                 generate_curves = True,
                                 n_jobs=-1,
                                 cv=5
                                )
run_stats_ga, curves_ga, cv_results_ga, best_est_ga = ga_nn.run()

In [ ]:
best_est_ga.best_params_

In [ ]:
%%time
ga_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'genetic_alg', 
    max_iters=100000, 
    learning_rate =  0.001,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    pop_size= 20,
    mutation_prob = 0.25,
)
train_df_ga = pd.DataFrame([])
val_df_ga = pd.DataFrame([])

for i in range(5):
    print(i)
    X_train_train, X_test_train, y_train_train, y_test_train = train_test_split(X_train_heart, y_train_heart,
                                                        test_size = 0.2, random_state = (i+1)*i, stratify = y_train_heart
                                                       )

    ga_nn_epochs.fit(X_train_train, y_train_train)
    train_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: train_curve}
    loss_df = pd.DataFrame(data=d)
    train_df_ga = pd.concat([train_df_ga,loss_df],1)
    ga_nn_epochs.fit(X_test_train, y_test_train)
    val_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: val_curve}
    loss_df = pd.DataFrame(data=d)
    val_df_ga = pd.concat([val_df_ga,loss_df],1)

In [ ]:
train_df_ga['Mean'] = train_df_ga.loc[:, [0,1,2,3,4]].mean(axis = 1)
val_df_ga['Mean'] = val_df_ga.loc[:, [0,1,2,3,4]].mean(axis = 1)
ax = plt.gca()
plt.plot(train_df_ga['Mean'],  label="Training Error")
plt.plot(val_df_ga['Mean'], label ='Validation Error')
plt.title('Loss Curves - '+ 'GA',fontsize = 16)
plt.xlabel("Epochs"), plt.ylabel("Loss Values"), plt.legend(loc="best",prop={'size': 14})
plt.show()

## Individual HPO - GA

### Population

In [ ]:
%%time
ga_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'genetic_alg', 
    max_iters=100000, 
    learning_rate =  0.001,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    pop_size= 20,
    mutation_prob = 0.25,
)
train_df_ga_pop_20 = pd.DataFrame([])
val_df_ga_pop_20 = pd.DataFrame([])

for i in range(5):
    print(i)
    X_train_train, X_test_train, y_train_train, y_test_train = train_test_split(X_train_heart, y_train_heart,
                                                        test_size = 0.2, random_state = (i+1)*i, stratify = y_train_heart
                                                       )

    ga_nn_epochs.fit(X_train_train, y_train_train)
    train_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: train_curve}
    loss_df = pd.DataFrame(data=d)
    train_df_ga_pop_20 = pd.concat([train_df_ga_pop_20,loss_df],1)
    ga_nn_epochs.fit(X_test_train, y_test_train)
    val_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: val_curve}
    loss_df = pd.DataFrame(data=d)
    val_df_ga_pop_20 = pd.concat([val_df_ga_pop_20,loss_df],1)

In [ ]:
%%time
ga_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'genetic_alg', 
    max_iters=100000, 
    learning_rate =  0.001,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    pop_size= 50,
    mutation_prob = 0.25,
)
train_df_ga_pop_50 = pd.DataFrame([])
val_df_ga_pop_50 = pd.DataFrame([])

for i in range(5):
    print(i)
    X_train_train, X_test_train, y_train_train, y_test_train = train_test_split(X_train_heart, y_train_heart,
                                                        test_size = 0.2, random_state = (i+1)*i, stratify = y_train_heart
                                                       )

    ga_nn_epochs.fit(X_train_train, y_train_train)
    train_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: train_curve}
    loss_df = pd.DataFrame(data=d)
    train_df_ga_pop_50 = pd.concat([train_df_ga_pop_50,loss_df],1)
    ga_nn_epochs.fit(X_test_train, y_test_train)
    val_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: val_curve}
    loss_df = pd.DataFrame(data=d)
    val_df_ga_pop_50 = pd.concat([val_df_ga_pop_50,loss_df],1)

In [ ]:
%%time
ga_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'genetic_alg', 
    max_iters=100000, 
    learning_rate =  0.001,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    pop_size= 100,
    mutation_prob = 0.25,
)
train_df_ga_pop_100 = pd.DataFrame([])
val_df_ga_pop_100 = pd.DataFrame([])

for i in range(5):
    print(i)
    X_train_train, X_test_train, y_train_train, y_test_train = train_test_split(X_train_heart, y_train_heart,
                                                        test_size = 0.2, random_state = (i+1)*i, stratify = y_train_heart
                                                       )

    ga_nn_epochs.fit(X_train_train, y_train_train)
    train_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: train_curve}
    loss_df = pd.DataFrame(data=d)
    train_df_ga_pop_100 = pd.concat([train_df_ga_pop_100,loss_df],1)
    ga_nn_epochs.fit(X_test_train, y_test_train)
    val_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: val_curve}
    loss_df = pd.DataFrame(data=d)
    val_df_ga_pop_100 = pd.concat([val_df_ga_pop_100,loss_df],1)

In [ ]:
%%time
ga_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'genetic_alg', 
    max_iters=100000, 
    learning_rate =  0.001,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    pop_size= 300,
    mutation_prob = 0.25,
)
train_df_ga_pop_300 = pd.DataFrame([])
val_df_ga_pop_300 = pd.DataFrame([])

for i in range(5):
    print(i)
    X_train_train, X_test_train, y_train_train, y_test_train = train_test_split(X_train_heart, y_train_heart,
                                                        test_size = 0.2, random_state = (i+1)*i, stratify = y_train_heart
                                                       )

    ga_nn_epochs.fit(X_train_train, y_train_train)
    train_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: train_curve}
    loss_df = pd.DataFrame(data=d)
    train_df_ga_pop_300 = pd.concat([train_df_ga_pop_300,loss_df],1)
    ga_nn_epochs.fit(X_test_train, y_test_train)
    val_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: val_curve}
    loss_df = pd.DataFrame(data=d)
    val_df_ga_pop_300 = pd.concat([val_df_ga_pop_300,loss_df],1)

In [ ]:
train_df_ga_pop_20['Mean'] = train_df_ga_pop_20.loc[:, [0,1,2,3,4]].mean(axis = 1)
val_df_ga_pop_20['Mean'] = val_df_ga_pop_20.loc[:, [0,1,2,3,4]].mean(axis = 1)

train_df_ga_pop_50['Mean'] = train_df_ga_pop_50.loc[:, [0,1,2,3,4]].mean(axis = 1)
val_df_ga_pop_50['Mean'] = val_df_ga_pop_50.loc[:, [0,1,2,3,4]].mean(axis = 1)

train_df_ga_pop_100['Mean'] = train_df_ga_pop_100.loc[:, [0,1,2,3,4]].mean(axis = 1)
val_df_ga_pop_100['Mean'] = val_df_ga_pop_100.loc[:, [0,1,2,3,4]].mean(axis = 1)

train_df_ga_pop_300['Mean'] = train_df_ga_pop_300.loc[:, [0,1,2,3,4]].mean(axis = 1)
val_df_ga_pop_300['Mean'] = val_df_ga_pop_300.loc[:, [0,1,2,3,4]].mean(axis = 1)

plt.plot(train_df_ga_pop_20['Mean'],  label="POP 20", color = 'blue')
#plt.plot(val_df_ga_pop_20['Mean'], '--',  label="POP 20 - Val", color = 'blue')
plt.plot(train_df_ga_pop_50['Mean'], label ='POP 50', color = 'orange')
#plt.plot(val_df_ga_pop_50['Mean'], '--',  label="POP 50 - Val", color = 'orange')
plt.plot(train_df_ga_pop_100['Mean'], label ='POP 100', color = 'green')
#plt.plot(val_df_ga_pop_100['Mean'], '--',  label="POP 100 - Val", color = 'green')
plt.plot(train_df_ga_pop_300['Mean'], label ='POP 300', color = 'red')
#plt.plot(val_df_ga_pop_300['Mean'], '--',  label="POP 300 - Val", color = 'red')

plt.xlabel("Epochs"), plt.ylabel("Loss Values"), plt.legend(loc="best",prop={'size': 14})
plt.title('Varying Population - Fitness Scores', size = 16)
plt.plot()

### Mutation Rate

In [ ]:
%%time
ga_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'genetic_alg', 
    max_iters=100000, 
    learning_rate =  0.001,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    pop_size= 20,
    mutation_prob = 0.1,
)
train_df_ga_mut_01 = pd.DataFrame([])
val_df_ga_mut_01 = pd.DataFrame([])

for i in range(5):
    print(i)
    X_train_train, X_test_train, y_train_train, y_test_train = train_test_split(X_train_heart, y_train_heart,
                                                        test_size = 0.2, random_state = (i+1)*i, stratify = y_train_heart
                                                       )

    ga_nn_epochs.fit(X_train_train, y_train_train)
    train_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: train_curve}
    loss_df = pd.DataFrame(data=d)
    train_df_ga_mut_01 = pd.concat([train_df_ga_mut_01,loss_df],1)
    ga_nn_epochs.fit(X_test_train, y_test_train)
    val_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: val_curve}
    loss_df = pd.DataFrame(data=d)
    val_df_ga_mut_01 = pd.concat([val_df_ga_mut_01,loss_df],1)

In [ ]:
%%time
ga_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'genetic_alg', 
    max_iters=100000, 
    learning_rate =  0.001,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    pop_size= 20,
    mutation_prob = 0.25,
)
train_df_ga_mut_025 = pd.DataFrame([])
val_df_ga_mut_025 = pd.DataFrame([])

for i in range(5):
    print(i)
    X_train_train, X_test_train, y_train_train, y_test_train = train_test_split(X_train_heart, y_train_heart,
                                                        test_size = 0.2, random_state = (i+1)*i, stratify = y_train_heart
                                                       )

    ga_nn_epochs.fit(X_train_train, y_train_train)
    train_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: train_curve}
    loss_df = pd.DataFrame(data=d)
    train_df_ga_mut_025 = pd.concat([train_df_ga_mut_025,loss_df],1)
    ga_nn_epochs.fit(X_test_train, y_test_train)
    val_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: val_curve}
    loss_df = pd.DataFrame(data=d)
    val_df_ga_mut_025 = pd.concat([val_df_ga_mut_025,loss_df],1)

In [ ]:
%%time
ga_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'genetic_alg', 
    max_iters=100000, 
    learning_rate =  0.001,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    pop_size= 20,
    mutation_prob = 0.5,
)
train_df_ga_mut_050 = pd.DataFrame([])
val_df_ga_mut_050 = pd.DataFrame([])

for i in range(5):
    print(i)
    X_train_train, X_test_train, y_train_train, y_test_train = train_test_split(X_train_heart, y_train_heart,
                                                        test_size = 0.2, random_state = (i+1)*i, stratify = y_train_heart
                                                       )

    ga_nn_epochs.fit(X_train_train, y_train_train)
    train_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: train_curve}
    loss_df = pd.DataFrame(data=d)
    train_df_ga_mut_050 = pd.concat([train_df_ga_mut_050,loss_df],1)
    ga_nn_epochs.fit(X_test_train, y_test_train)
    val_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: val_curve}
    loss_df = pd.DataFrame(data=d)
    val_df_ga_mut_050 = pd.concat([val_df_ga_mut_050,loss_df],1)

In [ ]:
%%time
ga_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'genetic_alg', 
    max_iters=100000, 
    learning_rate =  0.001,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    pop_size= 20,
    mutation_prob = 0.8,
)
train_df_ga_mut_080 = pd.DataFrame([])
val_df_ga_mut_080 = pd.DataFrame([])

for i in range(5):
    print(i)
    X_train_train, X_test_train, y_train_train, y_test_train = train_test_split(X_train_heart, y_train_heart,
                                                        test_size = 0.2, random_state = (i+1)*i, stratify = y_train_heart
                                                       )

    ga_nn_epochs.fit(X_train_train, y_train_train)
    train_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: train_curve}
    loss_df = pd.DataFrame(data=d)
    train_df_ga_mut_080 = pd.concat([train_df_ga_mut_080,loss_df],1)
    ga_nn_epochs.fit(X_test_train, y_test_train)
    val_curve = ga_nn_epochs.fitness_curve[:,0]
    d = {i: val_curve}
    loss_df = pd.DataFrame(data=d)
    val_df_ga_mut_080 = pd.concat([val_df_ga_mut_080,loss_df],1)

In [ ]:
train_df_ga_mut_01['Mean'] = train_df_ga_mut_01.loc[:, [0,1,2,3,4]].mean(axis = 1)
val_df_ga_mut_01['Mean'] = val_df_ga_mut_01.loc[:, [0,1,2,3,4]].mean(axis = 1)

train_df_ga_mut_025['Mean'] = train_df_ga_mut_025.loc[:, [0,1,2,3,4]].mean(axis = 1)
val_df_ga_mut_025['Mean'] = val_df_ga_mut_025.loc[:, [0,1,2,3,4]].mean(axis = 1)

train_df_ga_mut_050['Mean'] = train_df_ga_mut_050.loc[:, [0,1,2,3,4]].mean(axis = 1)
val_df_ga_mut_050['Mean'] = val_df_ga_mut_050.loc[:, [0,1,2,3,4]].mean(axis = 1)

train_df_ga_mut_080['Mean'] = train_df_ga_mut_080.loc[:, [0,1,2,3,4]].mean(axis = 1)
val_df_ga_mut_080['Mean'] = val_df_ga_mut_080.loc[:, [0,1,2,3,4]].mean(axis = 1)

plt.plot(train_df_ga_mut_01['Mean'],  label="MUT 0.10")
plt.plot(train_df_ga_mut_025['Mean'], label ='MUT 0.25')
plt.plot(train_df_ga_mut_050['Mean'], label ='MUT 0.50')
plt.plot(train_df_ga_mut_080['Mean'], label ='MUT 0.80')

plt.xlabel("Epochs"), plt.ylabel("Loss Values"), plt.legend(loc="best",prop={'size': 14})
plt.title('Varying Mutation - Fitness Scores', size = 16)

plt.plot()

In [ ]:
ga_train = ga_nn_epochs.fit(X_train, y_train)
y_pred = ga_train.predict(X_test_heart)
cm = confusion_matrix(y_test_heart, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=['No','Yes'],
                             )
disp.plot()
disp.im_.colorbar.remove()
plt.title('GA - Confusion Matrix')
plt.show()

## Simulated Annealing

In [ ]:
%%time

grid_search_sa = {
    "learning_rate_init": [0.0001,0.001,0.01,0.1,0.5],
    "activation": [mlrose_hiive.neural.activation.relu],
    "schedule":[mlrose_hiive.GeomDecay(1), mlrose_hiive.GeomDecay(50), mlrose_hiive.GeomDecay(100),
                 mlrose_hiive.GeomDecay(250), mlrose_hiive.GeomDecay(300),
                 mlrose_hiive.ExpDecay(1), mlrose_hiive.ExpDecay(50), mlrose_hiive.ExpDecay(100),
                 mlrose_hiive.ExpDecay(250), mlrose_hiive.ExpDecay(300)
                                          ],
}

sa_nn = mlrose_hiive.NNGSRunner(X_train_heart, y_train_heart, 
                                 X_test_heart, y_test_heart, 
                                 experiment_name= "NeuralNetworkSA",
                                 output_directory="nn_sa_algorithm/",
                                 seed=42, iteration_list=[100000], 
                                 algorithm=mlrose_hiive.algorithms.sa.simulated_annealing,
                                 hidden_layer_sizes=[[100, 100]],
                                 grid_search_parameters=grid_search_sa,
                                 grid_search_scorer_method=f1_score,
                                 early_stopping = True,
                                 max_attempts =100,
                                 generate_curves = True,
                                 n_jobs=-1,
                                 cv=5
                                )
run_stats_sa, curves_sa, cv_results_sa, best_est_sa = sa_nn.run()

In [ ]:
best_est_sa.best_params_

In [ ]:
%%time
sa_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'simulated_annealing', 
    max_iters=100000,
    bias = True,
    is_classifier = True, 
    learning_rate =  0.1,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    schedule=mlrose_hiive.GeomDecay(1)
)

time_start = time.time()
sa_train = sa_nn_epochs.fit(X_train_heart, y_train_heart)
fit_time = time.time()
print(f'fit_time = {fit_time-time_start}')
print(classification_report(y_test_heart, sa_nn_epochs.predict(X_test_heart)))
print(classification_report(y_train_heart, sa_nn_epochs.predict(X_train_heart)))
plt.plot(sa_nn_epochs.fitness_curve[:,0],label='Training')
sa_test = sa_nn_epochs.fit(X_test_heart, y_test_heart)
plt.plot(sa_nn_epochs.fitness_curve[:,0],label='Validation')
plt.legend()
fig=plt.gcf()
fig.set_size_inches(10,6)
plt.xlabel("Epoch", fontsize=18)
plt.ylabel("Loss", fontsize=18)

In [ ]:
y_pred = sa_train.predict(X_test_heart)
cm = confusion_matrix(y_test_heart, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=['No','Yes'],
                             )
disp.plot()
disp.im_.colorbar.remove()
plt.title('SA - Confusion Matrix')
plt.show()
print(classification_report(y_test_heart, y_pred))

In [ ]:
%%time
sa_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'simulated_annealing', 
    max_iters=100000,
    bias = True,
    is_classifier = True, 
    learning_rate =  0.1,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    schedule=mlrose_hiive.GeomDecay(1)
)

train_df_sa = pd.DataFrame([])
val_df_sa = pd.DataFrame([])

for i in range(5):
    print(i)
    X_train_train, X_test_train, y_train_train, y_test_train = train_test_split(X_train_heart, y_train_heart,
                                                        test_size = 0.2, random_state = (i+1)*i, stratify = y_train_heart
                                                       )

    sa_nn_epochs.fit(X_train_train, y_train_train)
    train_curve = sa_nn_epochs.fitness_curve[:,0]
    d = {i: train_curve}
    loss_df_train = pd.DataFrame(data=d)
    train_df_sa = pd.concat([train_df_sa,loss_df_train],1)
    sa_nn_epochs.fit(X_test_train, y_test_train)
    val_curve = sa_nn_epochs.fitness_curve[:,0]
    d = {i: val_curve}
    loss_df_val = pd.DataFrame(data=d)
    val_df_sa = pd.concat([val_df_sa,loss_df_val],1)

In [ ]:
train_df_sa['Mean'] = train_df_sa.loc[:, [0,1,2,3,4]].mean(axis = 1)
val_df_sa['Mean'] = val_df_sa.loc[:, [0,1,2,3,4]].mean(axis = 1)

ax = plt.gca()
plt.plot(train_df_sa['Mean'],  label="Training Error")
plt.plot(val_df_sa['Mean'], label ='Validation Error')
plt.title('Loss Curves - '+ 'SA',fontsize = 16)
plt.xlabel("Epochs"), plt.ylabel("Loss Values"), plt.legend(loc="best",prop={'size': 14})

plt.show()

## Individual HPO - SA

### Decay Schedule

In [ ]:
%%time
sa_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'simulated_annealing', 
    max_iters=100000,
    bias = True,
    is_classifier = True, 
    learning_rate =  0.1,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    schedule=mlrose_hiive.ExpDecay(1)
)

time_start = time.time()
sa_nn_epochs.fit(X_train_heart, y_train_heart)
fit_time = time.time()
print(f'fit_time = {fit_time-time_start}')
print(classification_report(y_test_heart, sa_nn_epochs.predict(X_test_heart)))
print(classification_report(y_train_heart, sa_nn_epochs.predict(X_train_heart)))
plt.plot(sa_nn_epochs.fitness_curve[:,0],label='Training')
sa_train_exp_1 = sa_nn_epochs.fitness_curve[:,0]
sa_nn_epochs.fit(X_test_heart, y_test_heart)
plt.plot(sa_nn_epochs.fitness_curve[:,0],label='Validation')
sa_test_exp_1 = sa_nn_epochs.fitness_curve[:,0]
plt.legend()
fig=plt.gcf()
fig.set_size_inches(10,6)
plt.xlabel("Epoch", fontsize=18)
plt.ylabel("Loss", fontsize=18)

In [ ]:
%%time
sa_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'simulated_annealing', 
    max_iters=100000,
    bias = True,
    is_classifier = True, 
    learning_rate =  0.1,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    schedule=mlrose_hiive.GeomDecay(1)
)

time_start = time.time()
sa_nn_epochs.fit(X_train_heart, y_train_heart)
fit_time = time.time()
print(f'fit_time = {fit_time-time_start}')
print(classification_report(y_test_heart, sa_nn_epochs.predict(X_test_heart)))
print(classification_report(y_train_heart, sa_nn_epochs.predict(X_train_heart)))
plt.plot(sa_nn_epochs.fitness_curve[:,0],label='Training')
sa_train_geo_1 = sa_nn_epochs.fitness_curve[:,0]
sa_nn_epochs.fit(X_test_heart, y_test_heart)
plt.plot(sa_nn_epochs.fitness_curve[:,0],label='Validation')
sa_test_geo_1 = sa_nn_epochs.fitness_curve[:,0]
plt.legend()
fig=plt.gcf()
fig.set_size_inches(10,6)
plt.xlabel("Epoch", fontsize=18)
plt.ylabel("Loss", fontsize=18)

In [ ]:
plt.plot(sa_train_geo_1,  label="Decay = Geo")
plt.plot(sa_train_exp_1, label ='Decay = Exp')


plt.xlabel("Epochs"), plt.ylabel("Loss Values"), plt.legend(loc="best",prop={'size': 14})
plt.title('Varying Schedules - Fitness Scores', size = 16)
plt.ylim([0.1,0.4])
plt.plot()

### Temperature

In [ ]:
%%time
sa_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'simulated_annealing', 
    max_iters=100000,
    bias = True,
    is_classifier = True, 
    learning_rate =  0.1,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    schedule=mlrose_hiive.GeomDecay(50)
)

time_start = time.time()
sa_nn_epochs.fit(X_train_heart, y_train_heart)
fit_time = time.time()
print(f'fit_time = {fit_time-time_start}')
print(classification_report(y_test_heart, sa_nn_epochs.predict(X_test_heart)))
print(classification_report(y_train_heart, sa_nn_epochs.predict(X_train_heart)))
plt.plot(sa_nn_epochs.fitness_curve[:,0],label='Training')
sa_train_geo_50 = sa_nn_epochs.fitness_curve[:,0]
sa_nn_epochs.fit(X_test_heart, y_test_heart)
plt.plot(sa_nn_epochs.fitness_curve[:,0],label='Validation')
sa_test_geo_50 = sa_nn_epochs.fitness_curve[:,0]
plt.legend()
fig=plt.gcf()
fig.set_size_inches(10,6)
plt.xlabel("Epoch", fontsize=18)
plt.ylabel("Loss", fontsize=18)

In [ ]:
%%time
sa_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'simulated_annealing', 
    max_iters=100000,
    bias = True,
    is_classifier = True, 
    learning_rate =  0.1,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    schedule=mlrose_hiive.GeomDecay(100)
)

time_start = time.time()
sa_nn_epochs.fit(X_train_heart, y_train_heart)
fit_time = time.time()
print(f'fit_time = {fit_time-time_start}')
print(classification_report(y_test_heart, sa_nn_epochs.predict(X_test_heart)))
print(classification_report(y_train_heart, sa_nn_epochs.predict(X_train_heart)))
plt.plot(sa_nn_epochs.fitness_curve[:,0],label='Training')
sa_train_geo_100 = sa_nn_epochs.fitness_curve[:,0]
sa_nn_epochs.fit(X_test_heart, y_test_heart)
plt.plot(sa_nn_epochs.fitness_curve[:,0],label='Validation')
sa_test_geo_100 = sa_nn_epochs.fitness_curve[:,0]
plt.legend()
fig=plt.gcf()
fig.set_size_inches(10,6)
plt.xlabel("Epoch", fontsize=18)
plt.ylabel("Loss", fontsize=18)

In [ ]:
%%time
sa_nn_epochs = mlrose_hiive.NeuralNetwork(
    hidden_nodes = [100,100], activation = 'relu', 
    algorithm = 'simulated_annealing', 
    max_iters=100000,
    bias = True,
    is_classifier = True, 
    learning_rate =  0.1,
    early_stopping = True,
    max_attempts = 100,
    curve=True,
    random_state=42,
    schedule=mlrose_hiive.GeomDecay(300)
)

time_start = time.time()
sa_nn_epochs.fit(X_train_heart, y_train_heart)
fit_time = time.time()
print(f'fit_time = {fit_time-time_start}')
print(classification_report(y_test_heart, sa_nn_epochs.predict(X_test_heart)))
print(classification_report(y_train_heart, sa_nn_epochs.predict(X_train_heart)))
plt.plot(sa_nn_epochs.fitness_curve[:,0],label='Training')
sa_train_geo_300 = sa_nn_epochs.fitness_curve[:,0]
sa_nn_epochs.fit(X_test_heart, y_test_heart)
plt.plot(sa_nn_epochs.fitness_curve[:,0],label='Validation')
sa_test_geo_300 = sa_nn_epochs.fitness_curve[:,0]
plt.legend()
fig=plt.gcf()
fig.set_size_inches(10,6)
plt.xlabel("Epoch", fontsize=18)
plt.ylabel("Loss", fontsize=18)

In [ ]:
plt.plot(sa_train_geo_1,  label="Temp = 1")
plt.plot(sa_train_geo_50, label ='Temp = 50')
plt.plot(sa_train_geo_100, label ='Temp = 100')
plt.plot(sa_train_geo_300, label ='Temp = 300')

plt.xlabel("Epochs"), plt.ylabel("Loss Values"), plt.legend(loc="best",prop={'size': 14})
plt.title('Varying Tempeature - Fitness Scores', size = 16)
plt.ylim([0.1,0.4])
plt.plot()

## MLP-ADAM

In [ ]:
pipeline_heart = MLPClassifier(random_state=42, activation = "relu",alpha =0.001, hidden_layer_sizes=[100,100],solver='adam')
train_df = pd.DataFrame([])
val_df = pd.DataFrame([])
for i in range(5):
    X_train_train, X_test_train, y_train_train, y_test_train = train_test_split(X_train_heart, y_train_heart,
                                                        test_size = 0.2, random_state = (i+1)*i, stratify = y_train_heart
                                                       )
    time_start = time.time()
    pipeline_heart.fit(X_train_train, y_train_train)
    fit_time = time.time()
    print(f'fit_time = {fit_time-time_start}')
    train_curve = pipeline_heart.loss_curve_
    d = {i: train_curve}
    loss_df = pd.DataFrame(data=d)
    train_df = pd.concat([train_df,loss_df],1)
    pipeline_heart.fit(X_test_train, y_test_train)
    val_curve = pipeline_heart.loss_curve_
    d = {i: val_curve}
    loss_df = pd.DataFrame(data=d)
    val_df = pd.concat([val_df,loss_df],1)

In [ ]:
%%time
train_df['Mean'] = train_df.loc[:, [0,1,2,3,4]].mean(axis = 1)
val_df['Mean'] = val_df.loc[:, [0,1,2,3,4]].mean(axis = 1)
ax = plt.gca()
plt.plot(train_df['Mean'],  label="Training Error")
plt.plot(val_df['Mean'], label ='Validation Error')
pipeline_heart.fit(X_test_heart, y_test_heart)
plt.plot(pipeline_heart.loss_curve_, label ='Test Error')
plt.title('Loss Curves - '+ 'Adam',fontsize = 16)
plt.xlabel("Epochs"), plt.ylabel("Loss Values"), plt.legend(loc="best",prop={'size': 14})

plt.show()

In [ ]:
pipeline_heart = MLPClassifier(random_state=42, activation = "relu",alpha =0.001, hidden_layer_sizes=[100,100],solver='adam')
pipeline_heart.fit(X_train_heart, y_train_heart)
y_pred = pipeline_heart.predict(X_test_heart)
cm = confusion_matrix(y_test_heart, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=['No','Yes'],
                             )
disp.plot()
disp.im_.colorbar.remove()
plt.title('ADAM - Confusion Matrix')
plt.show()
print(classification_report(y_test_heart, y_pred))

In [ ]:
ax = plt.gca()
plt.plot(train_df['Mean'],  label="Training Error")
plt.plot(val_df['Mean'], label ='Validation Error')
plt.title('Loss Curves - '+ 'Adam',fontsize = 16)
plt.xlabel("Epochs"), plt.ylabel("Loss Values"), plt.legend(loc="best",prop={'size': 14})

plt.show()

## Comparison

In [ ]:
ax = plt.gca()
plt.plot(train_df['Mean'],  label=" Training - Adam", color ='blue')
plt.plot(val_df['Mean'], '--',  label=" Val - Adam", color ='blue')
plt.plot(train_df_rhc['Mean'], label ='Training - RHC', color ='orange')
plt.plot(val_df_rhc['Mean'], '--', label ='Val - RHC', color ='orange')
plt.plot(train_df_ga['Mean'], label ='Training - GA', color ='green')
plt.plot(val_df_ga['Mean'], '--', label ='Val - GA', color ='green')
plt.plot(train_df_sa['Mean'], label ='Training - SA', color ='red')
plt.plot(val_df_sa['Mean'], '--', label ='Val - SA', color ='red')
plt.title('Loss Curves',fontsize = 16)
plt.xlabel("Epochs"), plt.ylabel("Loss Values"), plt.legend(loc="upper right",prop={'size': 12})
plt.xscale('log',base=10) 

plt.show()